In [7]:
!pip install rdkit-pypi
!pip install scikit-learn
!pip install joblib
!pip install imbalanced-learn

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib

data = pd.read_csv('/content/training-set.csv')

def smiles_to_fingerprint(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return list(fingerprint)
    else:
        return [0] * n_bits

fingerprints = data['SMILES_stand'].apply(smiles_to_fingerprint)
fingerprints_df = pd.DataFrame(fingerprints.tolist())

X = fingerprints_df
y = data['Label']

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

joblib.dump(scaler, 'scaler.pkl')
joblib.dump(rf_model, 'cytopathic_effect_rf_model.pkl')

y_pred = rf_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

from google.colab import files
files.download('scaler.pkl')
files.download('cytopathic_effect_rf_model.pkl')


Model Accuracy: 59.29%
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.64      0.59        64
           1       0.65      0.55      0.60        76

    accuracy                           0.59       140
   macro avg       0.60      0.60      0.59       140
weighted avg       0.60      0.59      0.59       140



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>